In [3]:
import numpy as np
import pandas as pd

In [14]:
def dot(W, x):
    value = np.dot(W, x)
    
    def vjp(u): # Same shape as ‘value’
        return np.outer(u, x), W.T.dot(u)

    return value, vjp


### Question 1: implement the relu function and its VJP in the format above. Using the finite difference equation (slide 13), make sure that the VJP is correct numerically.


In [6]:
#let's define epsilon
epsilon = 1e-6
x = np.array([4,5,6])
u = np.array([1,2,3])


def relu(x):
    value = np.maximum(0,x)
    
    def vjp(u):
        #computing relu derivation
        vjp_wrt_x = u*np.greater(x,0)
        return vjp_wrt_x,  # The comma is important! to return a tuple of size 1 and not a scalar

    return value, vjp

#---------------------------------------------------

def finite_dif(x):
    dif = (relu(x+epsilon)[0]-relu(x)[0])/epsilon
    return dif


value, vjp = relu(x)
dif = finite_dif(x)

print("Numerical check :", vjp(u) - u.T*dif )


Numerical check : [[-1.39777967e-10 -2.79555934e-10 -4.19333901e-10]]


The value of the difference between vjp and finite difference equation is in the order of $10^{-10}$, negligeable difference. We can considere it as correct numerically.

### Question 2: reusing dot and relu, implement a 2-layer MLP with a relu activation

In [7]:
def mlp2(x, W1, W2):
    value_W1,vjp_W1_x = dot(W1,x)       #return value,vjp
    value_relu,vjp_relu = relu(value_W1) #return value,vjp
    value,vjp_W2_relu = dot(W2,value_relu) #return value,vjp
    def vjp(u):
        vjp_wrt_W2,vjp_wrt_relu = vjp_W2_relu(u)
        val_vjp_relu, = vjp_relu(vjp_wrt_relu)
        vjp_wrt_W1, vjp_wrt_x  = vjp_W1_x(val_vjp_relu)
        return vjp_wrt_x, vjp_wrt_W1, vjp_wrt_W2 

    return value, vjp #output & vjp

#TEST#
#2 layers & 2 inputs
# W1 = np.array([[0.5,0.45],[0.40,0.35]])
# W2 = np.array([[0.30,0.25],[0.20,0.15]])

# x = np.array([0.4,0.6])

#2 layers & 2 inputs
x = np.array([1,2])
W1 = np.array([[6,8],[3,5]])
W2 = np.array([[1,8],[6,0]])



output, vjpmlp2 = mlp2(x, W1, W2)

#u = np.array([1,3])
#print(vjpmlp2(u))

We check the 2-layer MLP, with a 2 dimension input and output. (We will compute a final check later, after loss implementation)

In [69]:
output

array([126, 132])

### Question 3: implement the squared loss VJP

In [8]:
def squared_loss(y_pred, y):
    residual = y_pred - y
    
    def vjp(u):
        vjp_y_pred = u*(residual)
        vjp_y =u*(-residual)
        return vjp_y_pred, vjp_y

    value = 0.5 * np.sum(residual ** 2)
    # The code requires every output to be an array.
    return np.array([value]), vjp

### Question 4 : implement the loss by composing mlp2 and squared_loss

In [9]:
def loss(x, y, W1, W2):
  output_mlp2, vjp_mlp2 = mlp2(x, W1, W2)
  value, vjp_sqloss = squared_loss(output_mlp2, y)
  def vjp(u):
    vjp_wrt_output, vjp_wrt_y = vjp_sqloss(u)
    vjp_wrt_x, vjp_wrt_W1, vjp_wrt_W2 = vjp_mlp2(vjp_wrt_output)
    return  vjp_wrt_x, vjp_wrt_y, vjp_wrt_W1, vjp_wrt_W2

  return value, vjp

In [10]:
mlp2(x, W1, W2)[0]

array([126, 132])

We will now check the MLP2 with square_loss numerically : (check question 2 and 3)

In [38]:
def finite_dif(x,y,W1,W2,epsilon):
    x_grad = x.copy()
    y_grad = y.copy()
    W1_grad = W1.copy()
    W2_grad = W2.copy()

    for i in range(x.shape[0]):
            x_perturb=x.copy()
            x_perturb[i]+=epsilon
            x_grad[i]=((loss(x_perturb,y,W1,W2)[0]-loss(x,y,W1,W2)[0])/epsilon)
    
    for i in range(y.shape[0]):
            y_perturb=y.copy()
            y_perturb[i]+=epsilon
            y_grad[i]=((loss(x,y_perturb,W1,W2)[0]-loss(x,y,W1,W2)[0])/epsilon)

    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1_perturb=W1.copy()
            W1_perturb[i,j]+=epsilon
            W1_grad[i,j]=((loss(x,y,W1_perturb,W2)[0]-loss(x,y,W1,W2)[0])/epsilon)

    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_perturb=W2.copy()
            W2_perturb[i,j]+=epsilon
            W2_grad[i,j]=((loss(x,y,W1,W2_perturb)[0]-loss(x,y,W1,W2)[0])/epsilon)
            
    return(x_grad,y_grad,W1_grad,W2_grad)


#let's define epsilon
# epsilon = 1e-6
# x = np.array([1.,2.])
# #u = np.array([[1.,2.]])
# y = np.array([1.,2.])
# W1 = np.array([[0.3,0.35],[0.4,0.45]])
# W2 = np.array([[0.5,0.55],[0.6,0.65]])
u = np.array([1.])

x  = np.random.uniform(-10, 10, size=2)
#W  = np.random.uniform(size=(6,9))
W1 = np.random.uniform(size=(2,2))
W2 = np.random.uniform(size=(2,2))
#y_pred = np.random.uniform(size=1)
y   = np.random.randint(2, size=1).astype(float)
epsilon   = 1e-10

condition4grad2beOK = 1e-5

value, vjp = loss(x,y,W1,W2)

grad = finite_dif(x,y,W1,W2,epsilon)


# x_dif = vjp(u)[0] - u.T*grad[0]
# print("Numerical check on x : ", x_dif)
# if (np.all(x_dif) <= condition4grad2beOK) : print("test OK")
# print('---------------------------------------')
# y_dif = vjp(u)[1] - u.T*grad[1]
# print("Numerical check : ", y_dif ) 
# if (np.all(y_dif) <= condition4grad2beOK) : print("test OK")
# print('---------------------------------------')
# W1_dif = vjp(u)[2] - u.T*grad[2]
# print("Numerical check : ", W1_dif )
# if (np.all(W1_dif) <= condition4grad2beOK) : print("test OK")
# print('---------------------------------------')
# W2_dif = vjp(u)[3] - u.T*grad[3]
# print("Numerical check : ", W2_dif)
# if (np.all(W2_dif) <= condition4grad2beOK) : print("test OK")

Numerical check on x :  [0. 0.]
test OK
---------------------------------------
Numerical check :  [-1.e-10 -1.e-10]
---------------------------------------
Numerical check :  [[-0. -0.]
 [-0. -0.]]
test OK
---------------------------------------
Numerical check :  [[0. 0.]
 [0. 0.]]
test OK


The test is ok, it seems to work. 

In [41]:
def testVJP(func, *args):
    
    value, vjp = func(*args)
    u = np.random.random(value.shape)
    e   = 1e-10

    for k, [arg, vjp] in enumerate(zip(args, vjp(u))):
        
        shape = arg.shape
        num = np.zeros_like(arg)
        
        if len(shape)>1:
            
            for i in range(shape[0]):
                for j in range(shape[1]):
                    arg_e = np.copy(arg)
                    args_e = list(args)
                    arg_e[i, j] += e
                    args_e[k] = arg_e
                    num[i, j] = (func(*args_e)[0] - value)/e 
                    
        else:
            
            for i in range(shape[0]):
                    arg_e = np.copy(arg)
                    args_e = list(args)
                    arg_e[i] += e
                    args_e[k] = arg_e
                    num[i] = (func(*args_e)[0] - value)/e                    
                    
        num = np.array(num)
        print(f"VJP_var_{k} OK : ", np.all(np.abs(vjp - (u * num)) < condition4grad2beOK),
              f"(max diff : {np.max(np.abs(vjp - (u * num)))})")

In [102]:
u = np.random.random(value.shape)
type(u[0])

numpy.float64

In [42]:
condition4grad2beOK = 1e-5
testVJP(loss, x,y,W1,W2)

VJP_var_0 OK :  True (max diff : 0.0)
VJP_var_1 OK :  True (max diff : 5.524495245560156e-11)
VJP_var_2 OK :  True (max diff : 0.0)
VJP_var_3 OK :  True (max diff : 0.0)


### Question 5: implement an MLP with an arbitrary number of layers.

In [46]:
#W is the list of [W1,...,WN]
def mlpN(x, W):
    value_vjp_W = np.array([dot(W[0],x)],dtype=object) #initialisation value_W1,vjp_W1_relu
    vjps_relu = np.array([],dtype=object)
    N = W.shape[0] #nbr of layers
    for i in range(1,N): 
        #going through layers 2 until N 
        #print(i)
        value_relu,vjp_relu = relu(value_vjp_W[i-1][0])
        #value_vjp_relu = np.append(value_vjp_relu,[relu(value_vjp_W[i-1][0])],axis=0)    #value_relu,vjp_relu
        vjps_relu= np.append(vjps_relu,vjp_relu)
        value_vjp_W = np.append(value_vjp_W,[dot(W[i],value_relu)],axis=0)    #value_Wi,vjp_Wi_relu 
        
    def vjp(u):
        #init for layers N
        vjp_wrt_WN,vjp_wrt_relu = value_vjp_W[-1][1](u) #vjp_WN_relu(u)
        vjp_wrt_relu=np.array([vjp_wrt_relu]) 
        vjp_wrt_W=np.array([vjp_wrt_WN])

        for i in range(N-2,0,-1):
            #iteration on layers from N-1 to 2  
            val_vjp_relu, = vjps_relu[i][1](vjp_wrt_relu[-1]) #vjp_relu(vjp_wrt_relu)
            vjp_wrt_Wi,vjp_wrt_relu = value_vjp_W[i][1](val_vjp_relu)

            vjp_wrt_relu = np.append(vjp_wrt_relu,vjp_wrt_relu,axis=0)
            vjp_wrt = np.append(vjp_wrt_W,vjp_wrt_Wi,axis=0)

        #iteration on layers 1
        print(vjps_relu)
        val_vjp_relu, = vjps_relu[0][1](vjp_wrt_relu[-1])
        vjp_wrt_W1, vjp_wrt_x  = value_vjp_W[0][1](val_vjp_relu)
        vjp_wrt_W = np.append(vjp_wrt_W,vjp_wrt_W1,axis=0)
        vjp_wrt_W = np.append(vjp_wrt_W,vjp_wrt_x,axis=0)

        return np.flip(vjp_wrt_W)
        
    value = value_vjp_W[-1][0]
    return value, vjp #output & vjp

In [44]:
mlp2(x, W1, W2)[0]

array([0., 0.])

In [47]:
mlpN(x, np.array([W1, W2],dtype=object))[0]

array([0.0, 0.0], dtype=object)

We have the same result for both, the network weems to work.

### Question 6: implement SGD to train your MLP on a dataset of your choice. Study the impact of depth (number of layers) and width (number of hidden units).

dataset : https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009?resource=download

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import math
from numpy.linalg import norm
%matplotlib inline

In [50]:
data = pd.read_csv('./data/winequality-red.csv', sep=',')
data = data.drop_duplicates()
# copy the data
data_scaled = data.copy()
mean_data = data_scaled.mean()
std_data =  data_scaled.std()

# apply normalization techniques
for column in data_scaled.columns:
    if column!='quality':
        data_scaled[column] = (data_scaled[column] - mean_data[column]) / std_data[column]
    else :
        data_scaled[column] = (data_scaled[column] - mean_data[column])


df_label = data_scaled['quality']
df_features = data_scaled.drop(columns=['quality']) 

X_train, X_test, y_train, y_test = train_test_split(df_features, df_label, test_size=0.33, random_state=42, shuffle = True)

A_test = X_test.to_numpy() #training matrix
Y_test = y_test.to_numpy() #label matrix
A_train = X_train.to_numpy() #training matrix
Y_train = y_train.to_numpy() #label matrix
n, p = X_train.shape #number of feature

#x0 = np.random.rand(p).T #random initial point 
x0 = np.zeros((p))

In [51]:
def squared_loss(y_pred, y):
    residual = y_pred - y
    
    def vjp(u):
        vjp_y_pred = u*(residual)
        vjp_y = u*(-residual)
        return vjp_y_pred, vjp_y

    value = 0.5 * np.sum(residual ** 2)
    # The code requires every output to be an array.
    return np.array([value]), vjp

def loss(x, y, W):
  output_mlp, vjp_mlp = mlpN(x, W)
  value, vjp_sqloss = squared_loss(output_mlp, y)
  def vjp(u):
    vjp_wrt_output, vjp_wrt_y = vjp_sqloss(u)
    vjp_wrt_var = vjp_mlp(vjp_wrt_output)
    vjp_wrt_var.append(vjp_wrt_y)
    return  vjp_wrt_var

  return value, vjp

In [58]:
W

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])

In [64]:
np.dot(W.T,A_train.T)

array([[ 0.09082838,  4.40509777,  0.1339666 , ..., -4.48261231,
        -4.31592649, -4.56527795],
       [ 0.09082838,  4.40509777,  0.1339666 , ..., -4.48261231,
        -4.31592649, -4.56527795]])

In [109]:
type(error[0][0][0])

numpy.int32

In [115]:
dtype=np.float64(error)
type(dtype[0][1][1])


numpy.float64

In [122]:
X_train

array([9, 8])

In [124]:
type(A_train[0][0])

numpy.float64

In [126]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#W1 = np.array([[6,8],[3,5]])
#W2 = np.array([[1,8],[6,0]])

# Initialize the weight matrices for each layer
#W = np.array([W1,W2])
#print(len(W_list))
W = np.ones([2,11])
X_train = np.array([9,8])
y_train = np.array([1,7])

# Define the number of iterations and the learning rate
num_iterations = 100
learning_rate = 0.1
loss_list=[]
for i in range(num_iterations):

    #  forward pass 
    output, vjp = mlpN(W, A_train)
    error = output - y_train
    if i % 10 == 0:
        print(f'iteration: {i}, loss: {np.mean(error ** 2)}')
    loss_list.append(np.mean(error ** 2))
    
    # gradients
    vjp_x, vjp_W = vjp(error)

    # weights update
    for i in range(len(W)):
        W[i] = W[i] - learning_rate * vjp_W[i]



plt.figure(figsize=(16,5))
plt.rcParams['figure.dpi'] = 227
plt.plot(range(len(loss_list)), loss_list, c= 'darkorchid', lw=1)
plt.title('Mean square error', fontSize=14)
plt.xlabel('Iterations', fontSize=11)
plt.ylabel('MSE', fontSize=11)
plt.show()

ValueError: shapes (11,) and (2,11) not aligned: 11 (dim 0) != 2 (dim 0)

In [52]:
def SGD(x0, A, y, epochs = 50, nb=1 ,learning_rate = 0.01, stopping_threshold = 1e-6):
    # Initializing learning rate and iterations
    learning_rate = learning_rate #step size
    losses = [] #to return 
    vector=x0.copy() #starting point
    n,p = A.shape #nb of data point
    u = np.array([1.])
    # Estimation of optimal parameters
    for _ in range(epochs):
        print(_)
        ik = [int(el) for el in np.floor(np.random.rand(nb)*n)]
        # Calculating a new prediction
        #prediction = A[ik,:].dot(vector)

        #mean square error
        #error = prediction - y[ik] 
        #mse = 1/(2*nb)*(np.dot(error.T,error))
        error, vjp = loss(A[ik,:], y[ik], vector)
        print(error)
        losses.append(error)

        #calcultating gradient
        #grad_k = (1/nb)*A[ik,:].transpose().dot(error)
        grad_k = vjp(u)
        print(grad_k)
        #gradient descent step
        vector = vector - (learning_rate * grad_k)

    return vector, loss

In [91]:
#initialize the weight matrices for each layer
W = np.zeros([1,11,2])

iteration=100
lr = 0.1

for i in range(iteration):
    #forward pass
    output, vjp = mlpN(A_train,W)

### plot debug

In [105]:
plt.figure(figsize=(5, 5))
#x0 = np.zeros((p))
x0 = np.zeros([1,11,1])
final_x , loss = SGD(x0, A_train , Y_train)
plt.plot(loss)


plt.title('Mean square Error, stepsize at 0.01')
plt.xlabel('No. of iterations')
plt.ylabel('MSE')
plt.legend()
plt.show()

0
[23.50083438]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[]


IndexError: index 0 is out of bounds for axis 0 with size 0

<Figure size 500x500 with 0 Axes>

I didn't succeed this part, I had troubles on the implementation, mainly about simension during vjp calculation.